# Download CAMELS-US metadata data
Metadata consists of two files:
- `camels_attributes_v2.0.xlsx`: description of all headers used in `.txt` files
- `camels_topo.txt`: topograhic data for each station

In [12]:
import sys
import shutil
import requests
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [13]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [18]:
# Get the required info from the config file
data_path = cs.read_from_config(config_file,'data_path')
shps_path = cs.read_from_config(config_file,'ref_shps_path')
meta_url  = cs.read_from_config(config_file,'us_camels_meta_url')
topo_url  = cs.read_from_config(config_file,'us_camels_topo_url')

### Make output folder

In [15]:
# Construct the download location
download_folder = Path(data_path) / shps_path / 'CAMELS-US'

In [16]:
# Make sure the download folder exists
download_folder.mkdir(parents=True, exist_ok=True)

### Get the data

In [20]:
# Retry settings
retries_max = 10

In [21]:
for file_url in [meta_url,topo_url]:
    
    # Make the file name
    file_name = file_url.split('/')[-1].strip() # Get the last part of the url, strip whitespace and characters
    
    # Make sure the connection is re-tried if it fails
    retries_cur = 1
    while retries_cur <= retries_max:
        try: 

            # Send a HTTP request to the server and save the HTTP response in a response object called resp
            # 'stream = True' ensures that only response headers are downloaded initially 
            # (and not all file contents too, which are 2GB+)
            with requests.get(file_url.strip(), stream=True) as response:

                # Decode the response
                response.raw.decode_content = True
                content = response.raw

                # Write to file
                with open(download_folder / file_name, 'wb') as data:
                    shutil.copyfileobj(content, data)

        except Exception as e:
            print('Error downloading ' + file_url + ' on try ' + str(retries_cur) + ' with error: ' + str(e))
            retries_cur += 1
            continue
        else:
            break 